In [1]:
%pip install gurobipy

In [2]:
import gurobipy as gp
from gurobipy import GRB

In [3]:
import math
from itertools import combinations

In [4]:
# Instance 
class Instance(object):
    width = 0
    n = 0
    dimensions = []
    def __init__(self, width, n, dimensions):
        self.width = width
        self.n = n
        self.dimensions = dimensions


# Read instances: 
def read_file(file_name):
    dimensions = []
    with open(file_name) as f:
        width = int(f.readline())                 # Width of the plate
        n = int(f.readline())                     # Number of blocks
        while True:
            line = f.readline()
            if not line: 
                break
            dimensions.append(line.split(" "))    # Dimensions of each plate
    for dim in dimensions:
        dim[0] = int(dim[0])
        dim[1] = int(dim[1])
    instance = Instance(width, n, dimensions)
    return instance

instance_file = "instances\ins-14.txt"
instance = read_file(instance_file)


In [5]:
instance.dimensions

[[3, 3],
 [3, 4],
 [3, 5],
 [3, 6],
 [3, 7],
 [3, 8],
 [3, 9],
 [3, 10],
 [3, 14],
 [3, 18],
 [4, 4],
 [4, 6],
 [4, 11],
 [5, 6],
 [5, 15]]

In [6]:
width = instance.width
n = instance.n
x_dims_init=[]
y_dims_init=[]

for x_dim,y_dim in instance.dimensions:
    x_dims_init.append(x_dim)
    y_dims_init.append(y_dim)
    
    
min_height = sum([x_dims_init[i] * y_dims_init[i] for i in range(len(x_dims_init))])//width    
max_height = math.ceil(sum(y_dims_init)/(instance.width//max(x_dims_init)))
dims_upper_bound = max(max(x_dims_init), max(y_dims_init))
M_x = width
M_y = max_height

In [7]:
min_height

21

In [8]:
m = gp.Model('VLSI')

Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-29


In [9]:
height = m.addVar(name = 'height', vtype = GRB.INTEGER, lb = min_height, ub = max_height)

In [10]:
x = m.addVars(list(range(len(x_dims_init))), name="x_coords", vtype = GRB.INTEGER, lb = 0, ub = width - min(x_dims_init))

In [11]:
y = m.addVars(list(range(len(y_dims_init))), name="y_coords", vtype = GRB.INTEGER, lb = 0, 
              ub = max_height - min(y_dims_init))

In [12]:
x_dims = m.addVars(list(range(len(x_dims_init))), name="x_dims", vtype = GRB.INTEGER, lb = 0, ub = dims_upper_bound)

In [13]:
y_dims = m.addVars(list(range(len(y_dims_init))), name="y_dims", vtype = GRB.INTEGER, lb = 0, ub = dims_upper_bound)

In [14]:
rots = m.addVars(list(range(len(x_dims_init))), name="rotations", vtype = GRB.BINARY)

In [15]:
ors = m.addVars([(i,j,k) for (i,j) in combinations(x,2) for k in range(4)], name='ors', vtype = GRB.BINARY)

In [16]:
ors

{(0, 1, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 3): <gurobi.Var *Awaiting Model Update*>,
 (0, 2, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 2, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 2, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 2, 3): <gurobi.Var *Awaiting Model Update*>,
 (0, 3, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 3, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 3, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 3, 3): <gurobi.Var *Awaiting Model Update*>,
 (0, 4, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 4, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 4, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 4, 3): <gurobi.Var *Awaiting Model Update*>,
 (0, 5, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 5, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 5, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 5, 3): <gurobi.Var *Awaiting Model Update*>,


In [17]:
m.addConstrs((x_dims[i] == (1-rots[i]) * x_dims_init[i] + rots[i] * y_dims_init[i] for i in list(range(len(x_dims)))), name = 'x_rots')

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>}

In [18]:
m.addConstrs((y_dims[i] == (1-rots[i]) * y_dims_init[i] + rots[i] * x_dims_init[i] for i in list(range(len(x_dims)))), name = 'y_rots')

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>}

In [19]:
m.addConstrs((x[i] + x_dims[i] <= width for i in list(range(len(x_dims)) )), name='x_constraints')

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>}

In [20]:
m.addConstrs((y[i] + y_dims[i] <= height for i in list(range(len(y_dims)) )), name='y_constraints')

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>}

In [21]:
m.addConstrs((ors.sum(i,j,'*') >= 1 for (i,j) in combinations(x,2)), name = 'at_least_one')

{(0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Updat

In [22]:
m.addConstrs((x[j] + x_dims[j] <= x[i]  + M_x * (1 - ors[i,j,0]) for (i,j) in combinations(x,2)), name = '0')

{(0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Updat

In [23]:
m.addConstrs((x[i] + x_dims[i] <= x[j] + M_x * (1 - ors[i,j,1]) for (i,j) in combinations(x,2)), name = '1')

{(0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Updat

In [24]:
m.addConstrs((y[j] + y_dims[j] <= y[i] + M_y * (1 - ors[i,j,2]) for (i,j) in combinations(x,2)), name = '2')

{(0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Updat

In [25]:
m.addConstrs((y[i] + y_dims[i] <= y[j] + M_y * (1 - ors[i,j,3]) for (i,j) in combinations(x,2)), name = '3')

{(0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Updat

In [26]:
m.setObjective(height, GRB.MINIMIZE)

In [27]:
m.write('RAP.lp')

In [28]:
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 585 rows, 496 columns and 2231 nonzeros
Model fingerprint: 0xa47bee41
Variable types: 0 continuous, 496 integer (435 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 3e+01]
  RHS range        [1e+00, 3e+01]
Presolve removed 32 rows and 32 columns
Presolve time: 0.00s
Presolved: 553 rows, 464 columns, 2113 nonzeros
Variable types: 0 continuous, 464 integer (433 binary)
Found heuristic solution: objective 32.0000000

Root relaxation: objective 2.100000e+01, 209 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   21.00000    0    6   32.00000   21.00000  34.4%     -    0s
H    0     0                      2

In [29]:
m.getVars()

[<gurobi.Var height (value 21.0)>,
 <gurobi.Var x_coords[0] (value 5.0)>,
 <gurobi.Var x_coords[1] (value -0.0)>,
 <gurobi.Var x_coords[2] (value 10.0)>,
 <gurobi.Var x_coords[3] (value 4.0)>,
 <gurobi.Var x_coords[4] (value 8.0)>,
 <gurobi.Var x_coords[5] (value 18.0)>,
 <gurobi.Var x_coords[6] (value 18.0)>,
 <gurobi.Var x_coords[7] (value 5.0)>,
 <gurobi.Var x_coords[8] (value 15.0)>,
 <gurobi.Var x_coords[9] (value -0.0)>,
 <gurobi.Var x_coords[10] (value 11.0)>,
 <gurobi.Var x_coords[11] (value 15.0)>,
 <gurobi.Var x_coords[12] (value -0.0)>,
 <gurobi.Var x_coords[13] (value -0.0)>,
 <gurobi.Var x_coords[14] (value -0.0)>,
 <gurobi.Var y_coords[0] (value 0.0)>,
 <gurobi.Var y_coords[1] (value 6.0)>,
 <gurobi.Var y_coords[2] (value 6.0)>,
 <gurobi.Var y_coords[3] (value 6.0)>,
 <gurobi.Var y_coords[4] (value -0.0)>,
 <gurobi.Var y_coords[5] (value 13.0)>,
 <gurobi.Var y_coords[6] (value 4.0)>,
 <gurobi.Var y_coords[7] (value 3.0)>,
 <gurobi.Var y_coords[8] (value 4.0)>,
 <gurobi.Va